# AutoImputation

This notebook demonstrates the functionality of the `autoimpute` module, which provides an automated approach to selecting and applying optimal imputation methods for missing data. Rather than manually testing different approaches, `autoimpute` evaluates multiple methods (tuning their hyperparameters to the specific dataset), identifies which performs best for your specific data, and applies it to generate high-quality imputations.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.datasets import load_diabetes
import warnings

# Set pandas display options to limit table width
pd.set_option("display.width", 600)
pd.set_option("display.max_columns", 10)
pd.set_option("display.expand_frame_repr", False)

from microimpute.comparisons.autoimpute import autoimpute
from microimpute.visualizations.plotting import method_comparison_results

Error importing in API mode: ImportError("dlopen(/Users/movil1/envs/pe/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so, 0x0002): Library not loaded: /Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib\n  Referenced from: <38886600-97A2-37BA-9F86-5263C9A3CF6D> /Users/movil1/envs/pe/lib/python3.11/site-packages/_rinterface_cffi_api.abi3.so\n  Reason: tried: '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file), '/Library/Frameworks/R.framework/Versions/4.5-arm64/Resources/lib/libRblas.dylib' (no such file)")
Trying to import in ABI mode.
/Users/movil1/envs/pe/lib/python3.11/site-packages/rpy2/rinterface/__init__.py:1185: UserWarning: Environment variable "PWD" redefined by R and overriding existing variable. Current: "/", R: "/Users/movil1/Desktop/PYTHONJOBS/PolicyEngine/micro

## Data preparation

This demonstration uses the diabetes dataset from scikit-learn. In real-world imputation scenarios, you would typically have a "donor" dataset with complete information for both predictor and target variables, and a "receiver" dataset that lacks some target variables that need to be imputed.

In [2]:
# Load the diabetes dataset
diabetes = load_diabetes()
diabetes_data = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

# Display the first few rows to understand the data structure
diabetes_data.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


For this demonstration, the diabetes dataset is split into donor and receiver portions. Part of the data is treated as the donor dataset with complete information, and another part as the receiver dataset with some variables that need imputation. `autoimpute` handles imputation of numerical, categorical and boolean variables, lifting constraints on the choice of data sets and variables.

In [3]:
# Split the data into donor and receiver portions
donor_indices = np.random.choice(
    len(diabetes_data), size=int(0.7 * len(diabetes_data)), replace=False
)
receiver_indices = np.array(
    [i for i in range(len(diabetes_data)) if i not in donor_indices]
)

donor_data = diabetes_data.iloc[donor_indices].reset_index(drop=True)
receiver_data = diabetes_data.iloc[receiver_indices].reset_index(drop=True)

# Define which variables we'll use as predictors and which we want to impute
predictors = ["age", "sex", "bmi", "bp"]
imputed_variables = ["s1", "s4"]

# For demonstration purposes, we'll remove the variables we want to impute from the receiver dataset
receiver_data_without_targets = receiver_data.drop(columns=imputed_variables)

print(f"Donor data shape: {donor_data.shape}")
print(f"Receiver data shape: {receiver_data_without_targets.shape}")
print(f"Predictors: {predictors}")
print(f"Variables to impute: {imputed_variables}")

Donor data shape: (309, 10)
Receiver data shape: (133, 8)
Predictors: ['age', 'sex', 'bmi', 'bp']
Variables to impute: ['s1', 's4']


## Running `autoimpute` 

Use the `autoimpute` function to automatically evaluate different imputation methods, select the best one, and generate imputations. The function handles all the complexity of model evaluation, selection, and application in a single call.

In [4]:
warnings.filterwarnings("ignore")

# Run the autoimpute process
model_imputations, imputed_data, fitted_models_dict, method_results_df = autoimpute(
    donor_data=donor_data,
    receiver_data=receiver_data_without_targets,
    predictors=predictors,
    imputed_variables=imputed_variables,
    tune_hyperparameters=False,  # enable automated hyperparameter tuning if desired
    k_folds=3,  # Number of cross-validation folds
)

print(
    f"Shape of receiver data before imputation: {receiver_data_without_targets.shape} \nShape of receiver data after imputation: {imputed_data.shape}"
)

Evaluating models:   0%|          | 0/4 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.045165061950683594s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Shape of receiver data before imputation: (133, 8) 
Shape of receiver data after imputation: (133, 10)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.8s finished


## Understanding the results

The `autoimpute` function returns four key objects that provide comprehensive information about the imputation process:

- `imputations`: A dictionary where keys are quantiles used for imputation and values are DataFrames containing the imputed values at each quantile
- `imputed_data`: The receiver dataset with imputed values integrated into it
- `fitted_model`: The best-performing imputation model, already fitted on the donor data
- `method_results_df`: A DataFrame with detailed performance metrics for all evaluated imputation methods

In [5]:
# Examine the comparative performance of different imputation methods
print("Cross-validation results for different imputation methods:")
method_results_df

Cross-validation results for different imputation methods:


,0.05,0.1,0.15,0.2,0.25,...,0.8,0.85,0.9,0.95,mean_loss
QRF,0.005708,0.008702,0.011653,0.014294,0.017038,...,0.015543,0.013186,0.010225,0.007407,0.016133
OLS,0.003877,0.006550,0.008912,0.011015,0.012736,...,0.013675,0.011716,0.009141,0.005682,0.012939
QuantReg,0.003557,0.006391,0.008910,0.011050,0.012739,...,0.013878,0.011899,0.009290,0.005564,0.012882
Matching,0.023121,0.023189,0.023257,0.023325,0.023393,...,0.024142,0.024210,0.024278,0.024346,0.023733


The table above provides a comprehensive view of how each imputation method performs across different quantiles. The 'mean_loss' column shows the average quantile loss across all quantiles for each method. Lower values indicate better performance, and `autoimpute` automatically selects the method with the lowest average loss.

In [6]:
# Identify which method was selected as the best performer
best_method = method_results_df["mean_loss"].idxmin()
print(f"Best performing method: {best_method}")
print(f"Average loss: {method_results_df.loc[best_method, 'mean_loss']:.4f}")

Best performing method: QuantReg
Average loss: 0.0129


## Visualizing method comparison

Visualize how different methods perform across quantiles provides insight into which methods are most appropriate for different parts of the distribution.

In [7]:
# Extract the quantiles used in the evaluation
quantiles = [q for q in method_results_df.columns if isinstance(q, float)]

comparison_viz = method_comparison_results(
    data=method_results_df,
    metric_name="Test Quantile Loss",
    data_format="wide",
)
fig = comparison_viz.plot(
    title="Autoimpute Method Comparison",
    show_mean=True,
)
fig.show()

The plot above illustrates how each imputation method performs across different quantiles of the distribution. Methods with consistently lower lines generally perform better overall.

In [8]:
comparison_viz.summary()

,Method,Mean Test Quantile Loss,Best Quantile,Best Test Quantile Loss,Worst Quantile,Worst Test Quantile Loss
2,QuantReg,0.012882,0.05,0.003557,0.55,0.017279
1,OLS,0.012939,0.05,0.003877,0.55,0.017406
0,QRF,0.016133,0.05,0.005708,0.50,0.022673
3,Matching,0.023733,0.05,0.023121,0.95,0.024346


By calling `summary` on the object returned by `method_comparison_results` function, you can get a summary of the imputation results, including the mean and standard deviation of the quantile loss for each method. This summary can help you understand the performance of different imputation methods in a more concise manner.

## Examining the imputed values

Now let us assess the actual imputed values generated by the best-performing method.

In [ ]:
# Examine imputed values at the median (0.5 quantile)
median_imputations = model_imputations["best_method"][0.5] # Extract the best imputations with the "best_method" key
print("Median imputed values:")
median_imputations.head()

Median imputed values:


,s1,s4
0,0.026284,0.033348
1,0.013539,0.037486
2,-0.034974,-0.056726
3,-0.001103,0.009459
4,-0.010864,-0.002840


In [11]:
# Look at the full receiver dataset with imputed values integrated
print("Receiver dataset with imputed values:")
imputed_data.head()

Receiver dataset with imputed values:


,age,sex,bmi,bp,s2,s3,s5,s6,s1,s4
0,0.085299,0.050680,0.044451,-0.005670,-0.034194,-0.032356,0.002861,-0.025930,0.026284,0.033348
1,0.041708,0.050680,0.061696,-0.040099,0.006202,-0.028674,-0.014960,0.011349,0.013539,0.037486
2,-0.096328,-0.044642,-0.083808,0.008101,-0.090561,-0.013948,-0.062917,-0.034215,-0.034974,-0.056726
3,0.005383,0.050680,-0.001895,0.008101,-0.015719,-0.002903,0.038394,-0.013504,-0.001103,0.009459
4,-0.052738,0.050680,-0.018062,0.080401,0.107662,-0.039719,0.036060,-0.042499,-0.010864,-0.002840


## Evaluating imputation quality

In this demonstration, since the receiver dataset was artificially created by removing variables from the original data, there exists the unique opportunity to evaluate the quality of our imputations by comparing them to the actual values.

In [12]:
# Visualize comparison between actual and imputed values
for var in imputed_variables:
    fig = go.Figure()

    # Plot actual values
    fig.add_trace(
        go.Scatter(
            x=receiver_data.index,
            y=receiver_data[var],
            mode="markers",
            name="Actual values",
            marker=dict(color="blue", size=8),
        )
    )

    # Plot imputed values
    fig.add_trace(
        go.Scatter(
            x=imputed_data.index,
            y=imputed_data[var],
            mode="markers",
            name="Imputed values",
            marker=dict(color="red", size=8),
        )
    )

    # Customize the plot appearance
    fig.update_layout(
        title=f"Comparison of actual vs imputed values for {var}",
        xaxis_title="Sample Index",
        yaxis_title=f"{var} Value",
        legend_title="Type",
        hovermode="closest",
    )

    fig.show()

The plots above show how well the imputed values (red) match the actual values (blue) that were removed from the receiver dataset. This visual comparison helps assess the quality of the imputations generated by the best-performing method.

## Advanced usage

### Custom models and hyperparameters

The `autoimpute` function allows for customization of both the models to evaluate and their hyperparameters. This flexibility enables adaptation to specific dataset characteristics and imputation requirements. The models that support hyperparameter specification and tuning are Matching and QRF.

In [15]:
from microimpute.models import *

# Specify a custom subset of models to evaluate
custom_models = [QRF, OLS, Matching]

# Specify custom hyperparameters for some models
custom_hyperparameters = {
    "QRF": {"n_estimators": 200, "max_depth": 10},
    "Matching": {"constrained": True},
}

# Then simply run autoimpute with custom models and hyperparameters
(
    advanced_model_imputations,
    advanced_imputed_data,
    advanced_fitted_models_dict,
    advanced_method_results,
) = autoimpute(
    donor_data=donor_data,
    receiver_data=receiver_data_without_targets,
    predictors=predictors,
    imputed_variables=imputed_variables,
    models=custom_models,
    hyperparameters=custom_hyperparameters,
    k_folds=3,
)

advanced_model_imputations["best_method"][0.5]

Evaluating models:   0%|          | 0/3 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.03073406219482422s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.5s finished


,s1,s4
0,0.026479,0.038979
1,0.018359,0.042584
2,-0.028546,-0.050729
3,0.003142,0.015120
4,-0.003561,0.006434
...,...,...
128,0.038828,0.057077
129,-0.012671,-0.035450
130,0.010586,-0.008467
131,-0.005178,-0.021635


### Comparison of imputed values across models

For comparing, not only performance through quantile loss, but also final results through the evaluation of imputed values, `autoimpute` supports setting the parameter `impute_all` to True so that it will not only perform impuation with the model chosen as the best performing but also all others. When set to True, this parameter ensures that `autoimpute`'s returned imputations dictionary and fitted models dictionary contain all other models in addition to the "best_method".

In [17]:
warnings.filterwarnings("ignore")

# Run the autoimpute process
model_imputations, imputed_data, fitted_models_dict, method_results_df = autoimpute(
    donor_data=donor_data,
    receiver_data=receiver_data_without_targets,
    predictors=predictors,
    imputed_variables=imputed_variables,
    tune_hyperparameters=False, 
    impute_all=True,
    k_folds=3,
)

print(f"Imputation results available for models: {model_imputations.keys()}")
print(f"The best performing model is: {fitted_models_dict['best_method'].__class__.__name__}")

Evaluating models:   0%|          | 0/4 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.042740821838378906s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.7s finished


Imputation results available for models: dict_keys(['best_method', 'QRF', 'OLS', 'Matching'])
The best performing model is: QuantRegResults
